# Create the Master Data Table

This file creates a master table which combines all of the various data sources contained in the aws covid data lake, as well as the google mobility data.
* https://aws.amazon.com/blogs/big-data/a-public-data-lake-for-analysis-of-covid-19-data/
* Additional safegraph census data columns are available for selection in 'safegraph_census.ipynb'

In [1]:
import pandas as pd, pickle, boto3

# Data Pipepline

## Helper Functions

In [2]:
# Before using the aws related functions here, be sure to run 'aws configure' in your 
# virtual environment shell to set your ID and secret key

def getS3Data(bucket, file_loc, csv=True):
    client = boto3.client('s3')
    obj = client.get_object(Bucket=bucket, Key=file_loc)
    if(csv):
        data = pd.read_csv(obj['Body'] , dtype=str)
    else:
        data = pd.read_json(obj['Body'])
    return data

In [3]:
def dataCleanUp(data, date_field, fips_field, case_field, death_field, suffix): 
    print("Shape: ", data.shape)
    data[fips_field] = data[fips_field].astype('str').apply(lambda x: x.zfill(5))
    data[date_field] = data[date_field].astype('datetime64')
    data[case_field] = data[case_field].astype('float64')
    print("Case control total: ", data[case_field].sum())
    data[death_field] = data[death_field].astype('float64')
    print("Death control total: ", data[death_field].sum())
    data = data.add_suffix(suffix)
    data['date'] = data[date_field + suffix]
    data = data.rename(str.lower, axis='columns') 
    return data

In [4]:
def getDataInfo(data, case_field, death_field):
    print("Dataframe Shape: ", data.shape)
    print("Number of States: ", len(data['state_code'].unique()))
    print("Number of Counties: ", len(data['fips_code'].unique()))
    print("Minimum Date: ", data['date'].min())
    print("Maximum Date: ", data['date'].max())
    print("Duplicate State-Fips-Date: ", 
          data.groupby(['state_code','fips_code', 'date']).size().reset_index().rename(columns={0:'count'})['count'].sum() 
          - data.shape[0])
    print("Null State Code: ", data[data.state_code.isnull()].size)
    print("Null County Code: ", data[data.fips_code.isnull()].size)
    print("Null Dates: ", enigma_agg_data[enigma_agg_data.date.isnull()].size)
    
    print("Case Control Total: ", data[case_field].sum())
    print("Death Control Total: ", data[death_field].sum())

## Static Data

### County reference data

In [5]:
print("--------------------")
print("Census Data")

--------------------
Census Data


In [6]:
# Get data
county_ref_data = getS3Data('covid19-lake', 'static-datasets/csv/CountyPopulation/County_Population.csv')

# Rename columns
county_ref_data.columns = ['big_fips', 'fips', 'county', 'state', 'population_2018']

# Fill leading zeros on fips code
county_ref_data['fips'] = county_ref_data['fips'].astype('str').apply(lambda x: x.zfill(5))

### State reference data

In [7]:
# Get data
state_ref_data = getS3Data('covid19-lake', 'static-datasets/csv/state-abv/states_abv.csv')

# Rename columns
state_ref_data.columns = ['state_name','state_code']

### Safegraph Open Census Data
* See notebook: 'safegraph_census.ipynb'

In [8]:
with open('safegraph_census.p', 'rb') as f:
    safegraph_census_data = pickle.load(f)

#### FIPS Reference Data

In [9]:
# Get data
with open('safegraph_ref_data_fips.p', 'rb') as f:
    safegraph_ref_data_fips = pickle.load(f)

#### Land area

In [10]:
# Get data
with open('safegraph_land.p', 'rb') as f:
    safegraph_land = pickle.load(f)

## COVID Cases and Deaths Timeseries Data

### Enigma Aggregation

#### Download data and cleanup

In [11]:
print("--------------------")
print("Enigma Aggregation")

--------------------
Enigma Aggregation


In [12]:
# Get data
enigma_agg_data = getS3Data('covid19-lake', 'enigma-aggregation/csv/us_counties/enigma_covid_19_us_counties.csv')

In [13]:
# Clean up
enigma_agg_data = dataCleanUp(enigma_agg_data, 'date', 'county_fips', 'cases','deaths','_ea')

Shape:  (583062, 10)
Case control total:  602329120.0
Death control total:  22406363.0


In [14]:
# Change a few fips codes
enigma_agg_data['fips_code'] = enigma_agg_data['county_fips_ea']
enigma_agg_data.loc[enigma_agg_data['fips_code'] == '00nan', 'fips_code'] = '00000'

In [15]:
# Attach 2 digit state code from state fips code
tmp_state = safegraph_ref_data_fips.groupby(['state_fips','state']).size().reset_index().rename(columns={0:'count'})
enigma_agg_data['state_fips_ea'] = enigma_agg_data['state_fips_ea'].astype('str').apply(lambda x: x.zfill(2))

lefton = ['state_fips_ea']
righton = ['state_fips']

enigma_agg_data = pd.merge(enigma_agg_data, tmp_state[['state_fips','state']], how='left', left_on=lefton, right_on=righton)

enigma_agg_data['state_code'] = enigma_agg_data['state']
enigma_agg_data['state_name'] = enigma_agg_data['state_name_ea']

enigma_agg_data = enigma_agg_data.drop(columns=['state', 'state_fips'])

#### Checks

In [16]:
getDataInfo(enigma_agg_data, 'cases_ea', 'deaths_ea')

Dataframe Shape:  (583062, 14)
Number of States:  55
Number of Counties:  3211
Minimum Date:  2020-01-21 00:00:00
Maximum Date:  2020-09-29 00:00:00
Duplicate State-Fips-Date:  0
Null State Code:  0
Null County Code:  0
Null Dates:  0
Case Control Total:  602329120.0
Death Control Total:  22406363.0


### Enigma Johns Hopkins University

#### Download data and basic summary

In [17]:
print("--------------------")
print("Enigma Johns Hopkins")

--------------------
Enigma Johns Hopkins


In [18]:
enigma_jh_data = getS3Data('covid19-lake', 'enigma-jhu-timeseries/csv/jhu_csse_covid_19_timeseries_merged.csv')

In [19]:
# Filter to only US
enigma_jh_data = enigma_jh_data[enigma_jh_data['iso2']=='US']

In [20]:
enigma_jh_data = dataCleanUp(enigma_jh_data, 'date', 'fips', 'confirmed', 'deaths', '_ejhu')

Shape:  (419379, 14)
Case control total:  62110114.0
Death control total:  3522063.0


In [21]:
# Create clean fips_code
enigma_jh_data['fips_code'] = enigma_jh_data['fips_ejhu']

In [22]:
# Add "Out of XX" counties to "Unknown"
jh_unknown_fips_counties = ['Out of AL', 'Out of AK',
       'Out of AZ', 'Out of AR', 'Out of CA', 'Out of CO', 'Out of CT',
       'Out of DE', 'Out of DC', 'Out of FL', 'Out of GA', 'Out of HI',
       'Out of ID', 'Out of IL', 'Out of IN', 'Out of IA', 'Out of KS',
       'Out of KY', 'Out of LA', 'Out of ME', 'Out of MD', 'Out of MA',
       'Out of MI', 'Out of MN', 'Out of MS', 'Out of MO', 'Out of MT',
       'Out of NE', 'Out of NV', 'Out of NH', 'Out of NJ', 'Out of NM',
       'Out of NY', 'Out of NC', 'Out of ND', 'Out of OH', 'Out of OK',
       'Out of OR', 'Out of PA', 'Out of RI', 'Out of SC', 'Out of SD',
       'Out of TN', 'Out of TX', 'Out of UT', 'Out of VT', 'Out of VA',
       'Out of WA', 'Out of WV', 'Out of WI', 'Out of WY']
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu'].isin(jh_unknown_fips_counties), 'fips_code'] = '00000'

In [23]:
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Dukes and Nantucket', 'fips_code'] = 'n0001'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Kansas City', 'fips_code'] = 'n0002'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Michigan Department of Corrections (MDOC)', 'fips_code'] = 'n0003'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Federal Correctional Institution (FCI)','fips_code'] = 'n0004'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Bear River', 'fips_code'] = 'n0005'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Central Utah', 'fips_code'] = 'n0006'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Southeast Utah', 'fips_code'] = 'n0007'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Southwest Utah', 'fips_code'] = 'n0008'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='TriCounty', 'fips_code'] = 'n0009'
enigma_jh_data.loc[enigma_jh_data['admin2_ejhu']=='Weber-Morgan', 'fips_code'] = 'n0010'

In [24]:
lefton = ['province_state_ejhu']
righton = ['state_name']

enigma_jh_data = pd.merge(
    enigma_jh_data, state_ref_data, 
    how='left', left_on=lefton, right_on=righton)

In [25]:
enigma_jh_data.loc[enigma_jh_data['province_state_ejhu']=='Diamond Princess', 'state_code'] = 'CA'
enigma_jh_data.loc[enigma_jh_data['province_state_ejhu']=='Grand Princess', 'state_code'] = 'CA'

enigma_jh_data.loc[enigma_jh_data['province_state_ejhu']=='Diamond Princess', 'state_name'] = 'California'
enigma_jh_data.loc[enigma_jh_data['province_state_ejhu']=='Grand Princess', 'state_name'] = 'California'

In [26]:
enigma_jh_data['date'] = enigma_jh_data['date_ejhu']

In [27]:
enigma_jh_data.head(2)

,uid_ejhu,fips_ejhu,iso2_ejhu,iso3_ejhu,code3_ejhu,admin2_ejhu,latitude_ejhu,longitude_ejhu,province_state_ejhu,country_region_ejhu,date_ejhu,confirmed_ejhu,deaths_ejhu,recovered_ejhu,date,fips_code,state_name,state_code
0,84001001,01001,US,USA,840,Autauga,32.53952745,-86.64408227,Alabama,US,2020-01-22,0.0,0.0,NaN,2020-01-22,01001,Alabama,AL
1,84001003,01003,US,USA,840,Baldwin,30.72774991,-87.72207058,Alabama,US,2020-01-22,0.0,0.0,NaN,2020-01-22,01003,Alabama,AL


#### Checks

In [28]:
getDataInfo(enigma_jh_data, 'confirmed_ejhu', 'deaths_ejhu')

Dataframe Shape:  (419379, 18)
Number of States:  51
Number of Counties:  3201
Minimum Date:  2020-01-22 00:00:00
Maximum Date:  2020-05-29 00:00:00
Duplicate State-Fips-Date:  0
Null State Code:  0
Null County Code:  0
Null Dates:  0
Case Control Total:  62110114.0
Death Control Total:  3522063.0


### Enigma New York Times

#### Download data and clean data

In [29]:
print("--------------------")
print("Enigma New York Times")

--------------------
Enigma New York Times


In [30]:
# Get data
enigma_nyt_data = getS3Data('covid19-lake', 'enigma-nytimes-data-in-usa/csv/us_county/us_county.csv')

In [31]:
# Clean up
enigma_nyt_data = dataCleanUp(enigma_nyt_data, 'date', 'fips', 'cases','deaths','_enyt')

Shape:  (129747, 6)
Case control total:  31616769.0
Death control total:  1652495.0


In [32]:
# Fix Fips codes
enigma_nyt_data['fips_code'] = enigma_nyt_data['fips_enyt']
enigma_nyt_data.loc[enigma_nyt_data['county_enyt']=='New York City', 'fips_code'] = 'NYC000'
enigma_nyt_data.loc[enigma_nyt_data['county_enyt']=='New York City', 'fips_code'] = 'KC0000'
enigma_nyt_data.loc[enigma_nyt_data['county_enyt']=='Unknown', 'fips_code'] = '00000'

In [33]:
# Add state
lefton = ['state_enyt']
righton = ['state_name']

enigma_nyt_data = pd.merge(
    enigma_nyt_data, state_ref_data, 
    how='left', left_on=lefton, right_on=righton)

In [34]:
# Fix for extra states
enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Puerto Rico', 'state_code'] = 'PR'
enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Virgin Islands', 'state_code'] = 'VI'
enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Guam', 'state_code'] = 'GU'
enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Northern Mariana Islands', 'state_code'] = 'MP'

enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Puerto Rico', 'state_name'] = 'Puerto Rico'
enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Virgin Islands', 'state_name'] = 'Virgin Islands'
enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Guam', 'state_name'] = 'Guam'
enigma_nyt_data.loc[enigma_nyt_data['state_enyt']=='Northern Mariana Islands', 'state_name'] = 'Northern Mariana Islands'

#### Checks

In [35]:
getDataInfo(enigma_nyt_data, 'cases_enyt', 'deaths_enyt')

Dataframe Shape:  (129747, 10)
Number of States:  55
Number of Counties:  2885
Minimum Date:  2020-01-21 00:00:00
Maximum Date:  2020-05-09 00:00:00
Duplicate State-Fips-Date:  0
Null State Code:  0
Null County Code:  0
Null Dates:  0
Case Control Total:  31616769.0
Death Control Total:  1652495.0


### Rearc New York Times

#### Download data and basic summary

In [36]:
print("--------------------")
print("Rearc New York Times")

--------------------
Rearc New York Times


In [37]:
# Get data
rearc_nyt_data = getS3Data('covid19-lake', 'rearc-covid-19-nyt-data-in-usa/csv/us-counties/us-counties.csv')

In [38]:
# Clean up
rearc_nyt_data = dataCleanUp(rearc_nyt_data, 'date', 'fips', 'cases','deaths','_rnyt')

Shape:  (975686, 6)
Case control total:  2378946443.0
Death control total:  57311702.0


In [39]:
# Edit fips codes
rearc_nyt_data['fips_code'] = rearc_nyt_data['fips_rnyt']
rearc_nyt_data.loc[rearc_nyt_data['county_rnyt']=='New York City', 'fips_code'] = 'NYC000'
rearc_nyt_data.loc[rearc_nyt_data['county_rnyt']=='New York City', 'fips_code'] = 'KC0000'
rearc_nyt_data.loc[rearc_nyt_data['county_rnyt']=='Unknown', 'fips_code'] = '00000'

In [40]:
# Attach state code
lefton = ['state_rnyt']
righton = ['state_name']

rearc_nyt_data = pd.merge(
    rearc_nyt_data, state_ref_data, 
    how='left', left_on=lefton, right_on=righton)

In [41]:
# Add some states codes and names

rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Puerto Rico', 'state_code'] = 'PR'
rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Virgin Islands', 'state_code'] = 'VI'
rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Guam', 'state_code'] = 'GU'
rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Northern Mariana Islands', 'state_code'] = 'MP'

rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Puerto Rico', 'state_name'] = 'Puerto Rico'
rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Virgin Islands', 'state_name'] = 'Virgin Islands'
rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Guam', 'state_name'] = 'Guam'
rearc_nyt_data.loc[rearc_nyt_data['state_rnyt']=='Northern Mariana Islands', 'state_name'] = 'Northern Mariana Islands'

#### Checks

In [42]:
getDataInfo(rearc_nyt_data, 'cases_rnyt','deaths_rnyt')

Dataframe Shape:  (975686, 10)
Number of States:  55
Number of Counties:  3221
Minimum Date:  2020-01-21 00:00:00
Maximum Date:  2021-01-28 00:00:00
Duplicate State-Fips-Date:  0
Null State Code:  0
Null County Code:  0
Null Dates:  0
Case Control Total:  2378946443.0
Death Control Total:  57311702.0


### USA Facts (CDC Affiliate)

#### Download data and basic summary

In [43]:
print("--------------------")
print("USA Facts - CDC")

--------------------
USA Facts - CDC


In [44]:
# Download the data
url="https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv"
data_confirmed = pd.read_csv(url)

In [45]:
# Download the data
url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv"
data_deaths = pd.read_csv(url)

In [46]:
# Download the data
url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv"
data_county_pop = pd.read_csv(url)

In [47]:
# Move the date column headers into a single column as rows
data_confirmed = data_confirmed.melt(
    id_vars=['countyFIPS','County Name','State','stateFIPS'], 
    var_name='Date', value_name='Confirmed')

data_deaths = data_deaths.melt(
    id_vars=['countyFIPS','County Name','State','stateFIPS'], 
    var_name='Date', value_name='Deaths')

In [48]:
# Join the raw tables together

# Part 1
lefton = ['countyFIPS', 'County Name', 'State', 'stateFIPS', 'Date']
righton = ['countyFIPS', 'County Name', 'State', 'stateFIPS', 'Date']

data = pd.merge(data_confirmed, data_deaths, how='left', left_on=lefton, right_on=righton)

# Part 2
lefton = ['countyFIPS', 'County Name', 'State']
righton = ['countyFIPS', 'County Name', 'State']

usafacts_cdc_data = pd.merge(data, data_county_pop, how='left', left_on=lefton, right_on=righton)

In [49]:
# Clean up
usafacts_cdc_data = dataCleanUp(usafacts_cdc_data, 'Date', 'countyFIPS', 'Confirmed','Deaths','_cdc')

Shape:  (1191735, 8)
Case control total:  2339714800.0
Death control total:  56405847.0


In [50]:
# Fips code clean up
usafacts_cdc_data['fips_code'] = usafacts_cdc_data['countyfips_cdc']
usafacts_cdc_data.loc[(usafacts_cdc_data['county name_cdc']=='Grand Princess Cruise Ship'), 'fips_code'] = '99999'

In [51]:
# State code
usafacts_cdc_data['state_code'] = usafacts_cdc_data['state_cdc']

In [52]:
# Attach State Name
lefton = ['state_code']
righton = ['state_code']

usafacts_cdc_data = pd.merge(
    usafacts_cdc_data, state_ref_data, 
    how='left', left_on=lefton, right_on=righton)

#### Checks

In [53]:
getDataInfo(usafacts_cdc_data, 'confirmed_cdc','deaths_cdc')

Dataframe Shape:  (1191735, 12)
Number of States:  51
Number of Counties:  3146
Minimum Date:  2020-01-22 00:00:00
Maximum Date:  2021-01-28 00:00:00
Duplicate State-Fips-Date:  0
Null State Code:  0
Null County Code:  0
Null Dates:  0
Case Control Total:  2339714800.0
Death Control Total:  56405847.0


### Google Mobility Data

#### Download data

In [54]:
print("--------------------")
print("Google Mobility")

--------------------
Google Mobility


In [98]:
# Get data
url = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"
data = pd.read_csv(url, dtype=str)

In [99]:
# Clean up
data = data[data['country_region_code'] == 'US']
data['date'] = data['date'].astype('datetime64')
google_data = data

#### State Level Table

In [57]:
# Create state table
google_state_data = google_data[
    (google_data['sub_region_1'].notnull()) &
    (~google_data['sub_region_2'].notnull())
]
google_state_data = google_state_data.add_suffix("_goog_st")

In [58]:
# Add state_code
lefton = ['sub_region_1_goog_st']
righton = ['state_name']

google_state_data = pd.merge(
    google_state_data, state_ref_data, 
    how='left', left_on=lefton, right_on=righton)

In [59]:
# Drop columns not needed
google_state_data['date'] = google_state_data['date_goog_st']
google_state_data = google_state_data.drop(columns=
                                             ['country_region_code_goog_st',
                                              'country_region_goog_st', 'sub_region_1_goog_st','sub_region_2_goog_st',
                                              'date_goog_st','state_name'])

#### County Level Table

In [60]:
# Create county table
google_county_data = google_data[
    (google_data['sub_region_1'].notnull()) &
    (google_data['sub_region_2'].notnull())
]
google_county_data = google_county_data.add_suffix("_goog_cnty")

In [61]:
# Add state code
lefton = ['sub_region_1_goog_cnty']
righton = ['state_name']

google_county_data = pd.merge(
    google_county_data, state_ref_data, 
    how='left', left_on=lefton, right_on=righton)

In [62]:
# Create state_county concatenation
google_county_data['state_county'] = google_county_data['state_code'] + google_county_data['sub_region_2_goog_cnty']

In [63]:
# Manual state-county name fixes
google_county_data.loc[google_county_data['state_county']=='AKAnchorage', 'state_county'] = 'AKAnchorage Municipality'
google_county_data.loc[google_county_data['state_county']=='AKBethel', 'state_county'] = 'AKBethel Census Area'
google_county_data.loc[google_county_data['state_county']=='AKFairbanks North Star', 'state_county'] = 'AKFairbanks North Star Borough'
google_county_data.loc[google_county_data['state_county']=='AKJuneau', 'state_county'] = 'AKJuneau City and Borough'
google_county_data.loc[google_county_data['state_county']=='AKKetchikan Gateway', 'state_county'] = 'AKKetchikan Gateway Borough'
google_county_data.loc[google_county_data['state_county']=='AKKodiak Island', 'state_county'] = 'AKKodiak Island Borough'
google_county_data.loc[google_county_data['state_county']=='AKMatanuska-Susitna', 'state_county'] = 'AKMatanuska-Susitna Borough'
google_county_data.loc[google_county_data['state_county']=='AKNorth Slope', 'state_county'] = 'AKNorth Slope Borough'
google_county_data.loc[google_county_data['state_county']=='AKSitka', 'state_county'] = 'AKSitka City and Borough'
google_county_data.loc[google_county_data['state_county']=='AKSoutheast Fairbanks', 'state_county'] = 'AKSoutheast Fairbanks Census Area'
google_county_data.loc[google_county_data['state_county']=='AKValdez-Cordova', 'state_county'] = 'AKValdez-Cordova Census Area'
google_county_data.loc[google_county_data['state_county']=='MDBaltimore', 'state_county'] = 'MDBaltimore city'

google_county_data.loc[google_county_data['state_county']=='MOSt. Louis', 'state_county'] = 'MOSt. Louis city'
google_county_data.loc[google_county_data['state_county']=='NMDoña Ana County', 'state_county'] = 'NMDona Ana County'
google_county_data.loc[google_county_data['state_county']=='VAAlexandria', 'state_county'] = 'VAAlexandria city'
google_county_data.loc[google_county_data['state_county']=='VABristol', 'state_county'] = 'VABristol city'
google_county_data.loc[google_county_data['state_county']=='VABuena Vista', 'state_county'] = 'VABuena Vista city'
google_county_data.loc[google_county_data['state_county']=='VACharlottesville', 'state_county'] = 'VACharlottesville city'
google_county_data.loc[google_county_data['state_county']=='VAChesapeake', 'state_county'] = 'VAChesapeake city'
google_county_data.loc[google_county_data['state_county']=='VAColonial Heights', 'state_county'] = 'VAColonial Heights city'
google_county_data.loc[google_county_data['state_county']=='VACovington', 'state_county'] = 'VACovington city'

google_county_data.loc[google_county_data['state_county']=='VADanville', 'state_county'] = 'VADanville city'
google_county_data.loc[google_county_data['state_county']=='VAEmporia', 'state_county'] = 'VAEmporia city'
google_county_data.loc[google_county_data['state_county']=='VAFairfax', 'state_county'] = 'VAFairfax city'
google_county_data.loc[google_county_data['state_county']=='VAFalls Church', 'state_county'] = 'VAFalls Church city'
google_county_data.loc[google_county_data['state_county']=='VAFranklin', 'state_county'] = 'VAFranklin city'
google_county_data.loc[google_county_data['state_county']=='VAFredericksburg', 'state_county'] = 'VAFredericksburg city'
google_county_data.loc[google_county_data['state_county']=='VAGalax', 'state_county'] = 'VAGalax city'
google_county_data.loc[google_county_data['state_county']=='VAHampton', 'state_county'] = 'VAHampton city'
google_county_data.loc[google_county_data['state_county']=='VAHarrisonburg', 'state_county'] = 'VAHarrisonburg city'
google_county_data.loc[google_county_data['state_county']=='VAHopewell', 'state_county'] = 'VAHopewell city'

google_county_data.loc[google_county_data['state_county']=='VALexington', 'state_county'] = 'VALexington city'
google_county_data.loc[google_county_data['state_county']=='VALynchburg', 'state_county'] = 'VALynchburg city'
google_county_data.loc[google_county_data['state_county']=='VAManassas', 'state_county'] = 'VAManassas city'
google_county_data.loc[google_county_data['state_county']=='VAManassas Park', 'state_county'] = 'VAManassas Park city'
google_county_data.loc[google_county_data['state_county']=='VAMartinsville', 'state_county'] = 'VAMartinsville city'
google_county_data.loc[google_county_data['state_county']=='VANewport News', 'state_county'] = 'VANewport News city'
google_county_data.loc[google_county_data['state_county']=='VANorfolk', 'state_county'] = 'VANorfolk city'
google_county_data.loc[google_county_data['state_county']=='VANorton', 'state_county'] = 'VANorton city'
google_county_data.loc[google_county_data['state_county']=='VAPetersburg', 'state_county'] = 'VAPetersburg city'
google_county_data.loc[google_county_data['state_county']=='VAPoquoson', 'state_county'] = 'VAPoquoson city'
google_county_data.loc[google_county_data['state_county']=='VAPortsmouth', 'state_county'] = 'VAPortsmouth city'
google_county_data.loc[google_county_data['state_county']=='VARadford', 'state_county'] = 'VARadford city'
google_county_data.loc[google_county_data['state_county']=='VARichmond', 'state_county'] = 'VARichmond city'
google_county_data.loc[google_county_data['state_county']=='VARoanoke', 'state_county'] = 'VARoanoke city'

google_county_data.loc[google_county_data['state_county']=='VASalem', 'state_county'] = 'VASalem city'
google_county_data.loc[google_county_data['state_county']=='VAStaunton', 'state_county'] = 'VAStaunton city'
google_county_data.loc[google_county_data['state_county']=='VASuffolk', 'state_county'] = 'VASuffolk city'
google_county_data.loc[google_county_data['state_county']=='VAVirginia Beach', 'state_county'] = 'VAVirginia Beach city'
google_county_data.loc[google_county_data['state_county']=='VAWaynesboro', 'state_county'] = 'VAWaynesboro city'
google_county_data.loc[google_county_data['state_county']=='VAWilliamsburg', 'state_county'] = 'VAWilliamsburg city'
google_county_data.loc[google_county_data['state_county']=='VAWinchester', 'state_county'] = 'VAWinchester city'

In [64]:
# Attach FIPS code
lefton = ['state_county']
righton = ['state_county']

google_county_data = pd.merge(
    google_county_data, safegraph_ref_data_fips, 
    how='left', left_on=lefton, right_on=righton)

In [65]:
# Create fips_code and data columns
google_county_data['fips_code'] = google_county_data['fips']
google_county_data['date'] = google_county_data['date_goog_cnty']

In [66]:
# List counties which did not get a fips code
list(google_county_data[google_county_data.fips.isnull()]['state_county'].unique())

['AKNome', 'SDOglala Lakota County']

In [67]:
# Add missing county manually
google_county_data.loc[google_county_data['state_county']=='SDOglala Lakota County', 'fips_code'] = 'SDOGL'

In [68]:
# Drop columns not needed
google_county_data = google_county_data.drop(columns=['country_region_code_goog_cnty',
       'country_region_goog_cnty', 'sub_region_1_goog_cnty',
      'date_goog_cnty','state_name',
       'state_county', 'state', 'state_fips', 'county_fips', 'county',
       'class_code', 'fips'])

## Combine Data Sources

### Merge Data

In [69]:
print("--------------------")
print("Joining Data")

--------------------
Joining Data


In [70]:
# Build a list of tables to join
raw_data_list = [
    enigma_agg_data,
    enigma_jh_data,
    enigma_nyt_data,
    rearc_nyt_data,
    usafacts_cdc_data
]

In [71]:
# Since we want the sum columns to have different names, put them in a list
sum_cols = [
    ['cases_ea', 'deaths_ea'],
    ['confirmed_ejhu', 'deaths_ejhu'],
    ['cases_enyt', 'deaths_enyt'],
    ['cases_rnyt', 'deaths_rnyt'],
    ['confirmed_cdc', 'deaths_cdc']
]

In [72]:
# Recursively group the tables and put that in a new list
grouped_data_list = []
for data, sum_col in zip(raw_data_list, sum_cols):
    data = data.groupby(['state_code','fips_code', 'date'])[sum_col].agg('sum').reset_index()
    grouped_data_list.append(data)

In [73]:
# Add google mobility county data
grouped_data_list.append(
    google_county_data
)

In [74]:
# Join all the tables together
on_col = ['state_code', 'fips_code', 'date']
covid_data = pd.DataFrame(columns=on_col)
for data in grouped_data_list:
    covid_data = pd.merge(
        covid_data, data, how='outer', left_on=on_col, right_on=on_col
    )

In [75]:
# Add google mobility state data
on_col = ['state_code', 'date']
covid_data = covid_data = pd.merge(covid_data, google_state_data, how='left', left_on=on_col, right_on=on_col)

### Attach County Name

In [76]:
# Create county name lookup table from all data used so far
cols = [
    ['county_name_ea'],
    ['admin2_ejhu'],
    ['county_enyt'],
    ['county_rnyt'],
    ['county name_cdc'],
    ['sub_region_2_goog_cnty']  
]

raw_data_list = [
    enigma_agg_data,
    enigma_jh_data,
    enigma_nyt_data,
    rearc_nyt_data,
    usafacts_cdc_data,
    google_county_data
]

county_data_list = []
for data, col in zip(raw_data_list, cols):
    data = data.groupby(['state_code','fips_code','date']+col).size().reset_index().drop(columns=[0])
    data = data.rename(columns={col[0]:"county_name"})
    county_data_list.append(data)
data = pd.concat(county_data_list).dropna(subset=['county_name']).drop(columns=['date'])
data = data.drop_duplicates(subset=['state_code','fips_code'], keep="first").reset_index(drop=True)
county_names = data

In [77]:
# Attach county name
on_col = ['state_code', 'fips_code']
covid_data = pd.merge(covid_data, county_names, how='left', left_on=on_col, right_on=on_col)

In [78]:
# Attach land area
on_col = ['fips_code']
covid_data = pd.merge(covid_data, safegraph_land, how='left', left_on=on_col, right_on=on_col)

In [79]:
# Drop extra google county name columns
covid_data = covid_data.drop(columns=['sub_region_2_goog_cnty'])

### Attach Lat and Long

In [80]:
# Create lat long lookup table
lat_long_data = enigma_agg_data.groupby(['state_code','fips_code','lat_ea','long_ea']).size().reset_index().drop(columns=[0])
lat_long_data = lat_long_data.dropna(subset=['lat_ea','long_ea'])
lat_long_data = lat_long_data.drop_duplicates(subset=['state_code','fips_code'], keep="first").reset_index(drop=True)

In [81]:
# Join lat long data
on_col = ['state_code', 'fips_code']
covid_data = pd.merge(covid_data, lat_long_data, how='left', left_on=on_col, right_on=on_col)

### Attach Safegraph Census Data

In [82]:
# Join safegraph census data
covid_data = pd.merge(covid_data, safegraph_census_data, how='outer', left_on='fips_code', right_on='fips_code')

### Feature Engineering

In [83]:
def createMobilityChange(df, cols_to_average, rolling_ave, new_col_name):
    for col in cols_to_average:
        df.loc[:, col] = df[col].astype(float)
    df[new_col_name] = df[cols_to_average].mean(axis=1).rolling(window=7).mean()
    return df

In [84]:
def addFeatures(df):
    df['pop_density'] = df['pop_total'] / df['amount_land']
    df['deaths_per_100k'] = df['deaths_cdc'] / df['pop_total'] * 100000
    df['pir_200_plus_pct'] = df['pir_200_plus'] / df['pir_total']
    df['pir_150_plus_pct'] = (df['pir_150_184'] + df['pir_185_199'] + df['pir_200_plus']) / df['pir_total']
    df['unins_pct'] = (df['unins_0_18'] + df['unins_18_34'] + df['unins_35_64'] + df['unins_65_plus'])/df['unins_total']
    df['hi_75_plus_pct'] = (df['hi_75_99'] + df['hi_100_124'] + df['hi_125_149'] + df['hi_150_199']+ df['hi_200_plus'])/df['hi_total']
    df['hi_100_plus_pct'] = (df['hi_100_124'] + df['hi_125_149'] + df['hi_150_199']+ df['hi_200_plus'])/df['hi_total']
    df['hi_150_plus_pct'] = (df['hi_125_149'] + df['hi_150_199']+ df['hi_200_plus'])/df['hi_total']
    df['e_bach_plus_pct'] = (df['e_bach'] + df['e_mast'] + df['e_prof'] + df['e_doct'])/df['e_total']
    df['e_mast_plus_pct'] = (df['e_mast'] + df['e_prof'] + df['e_doct'])/df['e_total']
    df['r_white_pct'] = df['r_white'] / df['r_total']
    df['r_black_pct'] = df['r_black'] / df['r_total']
    df['r_native_pct'] = df['r_native'] / df['r_total']
    df['r_asian_pct'] = df['r_asian'] / df['r_total']
    df['age_55_plus_pct'] = (
        (df['m_55_59']+df['m_60_61']+df['m_62_64']+df['m_65_66']+df['m_67_69']+df['m_70_74']
         +df['m_75_79']+df['m_80_84']+df['m_85_110']) +
         (df['f_55_59']+df['f_60_61']+df['f_62_64']+df['f_65_66']+df['f_67_69']+df['f_70_74']
         +df['f_75_79']+df['f_80_84']+df['f_85_110']))/ (df['m_total'] + df['f_total'])
        
    return df

In [85]:
def addPercentiles(df, percentile_cols):
    data = df
    data_list = []
    states = list(df.state_code.unique())
    for state in states:
        cols = []
        df = addFeatures(data)
        df = df[df.state_code == state]
        df = df.dropna(subset=['pop_total']).drop_duplicates(subset=['state_code','fips_code']).drop(columns=['date'])
        for percentile_col in percentile_cols:
            df = df.sort_values(by=[percentile_col])
            df['cum_pop_total'] = df['pop_total'].cumsum()
            df[percentile_col + '_percentile'] = df['cum_pop_total'] / df['pop_total'].sum()
        cols = [col + '_percentile' for col in percentile_cols]
        cols = ['state_code','fips_code'] + cols
        df = df[cols]
        data_list.append(df)
    df = pd.concat(data_list)
    data = pd.merge(data, df, how='left', left_on=['state_code','fips_code'], right_on=['state_code','fips_code'])
    return data

In [86]:
# Create the mobility change column as the average of retail and work spaces
cols_to_average = ['retail_and_recreation_percent_change_from_baseline_goog_cnty',
                   'workplaces_percent_change_from_baseline_goog_cnty']
covid_data = createMobilityChange(covid_data, cols_to_average, 14, 'mobility_change_cnty')

In [87]:
# Attach percentile columns
percentile_cols = ['p_c_i', 'pop_density','pir_200_plus_pct', 'pir_150_plus_pct', 'unins_pct',
                  'hi_75_plus_pct', 'hi_100_plus_pct', 'hi_150_plus_pct', 'e_bach_plus_pct', 'e_mast_plus_pct',
                   'r_white_pct','r_black_pct', 'r_native_pct', 'r_asian_pct','age_55_plus_pct']
covid_data = addPercentiles(covid_data, percentile_cols)

# Add low and high percentile columns
covid_data['pir_grp'] = [0 if x < 0.5 else 1 for x in covid_data['pir_200_plus_pct_percentile']]
covid_data['unins_grp'] = [0 if x < 0.5 else 1 for x in covid_data['unins_pct_percentile']]
covid_data['pop_density_grp'] = [0 if x < 0.5 else 1 for x in covid_data['pop_density_percentile']] 
covid_data['e_grp'] = [0 if x < 0.5 else 1 for x in covid_data['e_bach_plus_pct_percentile']] 
covid_data['r_b_grp'] = [0 if x < 0.5 else 1 for x in covid_data['r_black_pct_percentile']] 
covid_data['r_w_grp'] = [0 if x < 0.5 else 1 for x in covid_data['r_white_pct_percentile']]
covid_data['age_55_plus_grp'] = [0 if x < 0.5 else 1 for x in covid_data['age_55_plus_pct_percentile']]

### Save Master Table

In [88]:
print("--------------------")
print("Saving data to covid_data.p")

--------------------
Saving data to covid_data.p


In [89]:
with open('covid_data.p', 'wb') as f:
    pickle.dump(covid_data, f)

In [90]:
covid_data.head(2)

,state_code,fips_code,date,cases_ea,deaths_ea,confirmed_ejhu,deaths_ejhu,cases_enyt,deaths_enyt,cases_rnyt,...,r_native_pct_percentile,r_asian_pct_percentile,age_55_plus_pct_percentile,pir_grp,unins_grp,pop_density_grp,e_grp,r_b_grp,r_w_grp,age_55_plus_grp
0,AK,00000,2020-04-02,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,1,1,1,1,1,1,1
1,AK,00000,2020-04-08,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,1,1,1,1,1,1,1


### Control Totals

In [91]:
print("--------------------")
print("Control Totals")

--------------------
Control Totals


In [92]:
print("--------------------")
print("Census Data")

--------------------
Census Data


In [93]:
# Safegraph census data control
covid_data[covid_data['date']=='2020-05-10'].pop_total.sum() + covid_data[covid_data['date'].isnull()].pop_total.sum() - safegraph_census_data.pop_total.sum()

-7782.0

In [94]:
print("--------------------")
print("Cases")

--------------------
Cases


In [95]:
print(enigma_agg_data.cases_ea.sum() - covid_data.cases_ea.sum())
print(enigma_jh_data.confirmed_ejhu.sum() - covid_data.confirmed_ejhu.sum())
print(enigma_nyt_data.cases_enyt.sum() - covid_data.cases_enyt.sum())
print(rearc_nyt_data.cases_rnyt.sum() - covid_data.cases_rnyt.sum())
print(usafacts_cdc_data.confirmed_cdc.sum() - covid_data.confirmed_cdc.sum())

0.0
0.0
0.0
0.0
0.0


In [96]:
print("--------------------")
print("Deaths")

--------------------
Deaths


In [97]:
print(enigma_agg_data.deaths_ea.sum() - covid_data.deaths_ea.sum())
print(enigma_jh_data.deaths_ejhu.sum() - covid_data.deaths_ejhu.sum())
print(enigma_nyt_data.deaths_enyt.sum() - covid_data.deaths_enyt.sum())
print(rearc_nyt_data.deaths_rnyt.sum() - covid_data.deaths_rnyt.sum())
print(usafacts_cdc_data.deaths_cdc.sum() - covid_data.deaths_cdc.sum())

0.0
0.0
0.0
0.0
0.0
